This example models a gas station and cars that arrive at the station for refueling.

Covers:

* [Depletable Resources](../resource.md#)
* Process Interaction, in particular [waiting](../component.md#wait) for other processes

The gas station has a limited number of fuel pumps, and a fuel tank that is shared between the fuel pumps. The gas station is thus modeled as `Resource`. The shared fuel tank is modeled with a `DepletableResource`.

Vehicles arriving at the gas station first request a fuel pump from the station. Once they acquire one, they try to take the desired amount of fuel from the fuel pump. They leave when they are done.

The gas stations fuel level is regularly monitored by gas station control. When the level drops below a certain threshold, a tank truck is called to refuel the gas station itself.

In [43]:
//@file:Repository("*mavenLocal")
//@file:DependsOn("com.github.holgerbrandl:kalasim:0.7-SNAPSHOT")

@file:DependsOn("com.github.holgerbrandl:kalasim:0.7.90")
@file:DependsOn("com.github.holgerbrandl:kravis:0.8.1")

In [44]:
import org.kalasim.*

import kravis.geomBar
import kravis.geomCol
import kravis.plot
import org.apache.commons.math3.distribution.LogNormalDistribution

import java.lang.Double.min
import kotlin.random.Random

In [45]:
//LogNormalDistribution(3.5, 0.88).let{ dist ->
//    repeat(100){ println(dist.sample())}
//}

//TODO set random seed, blocked by https://github.com/Kotlin/kotlin-jupyter/issues/345


In [86]:
class SquidGame(
    val numSteps: Int = 18,
    val numPlayers: Int = 16,
    val maxDuration: Int = 12 * 60
) : Environment(randomSeed =Random.nextInt()) {

    // randomization
    val stepTime = LogNormalDistribution(rg, 3.0, 0.88)
//    val stepTime = uniform(10,30)

    val decision = enumerated(true, false)

    // state
    var stepsLeft = numSteps
    var survivors= mutableListOf<Int>()

    val numTrials: Int
        get() = numSteps - survivors.size

    val numSurvivors : Int
       get() = survivors.size

    fun playerSurvived(playerNo: Int) = survivors.contains(playerNo)

    init {
        object : Component() {
            override fun process() = sequence {
               queue@
               for(player in 1..numPlayers){
                    hold(min(stepTime(), 100.0)) // cap time at 100sec

                    while(stepsLeft-- > 0){
                        if(decision()) continue@queue
                        hold(min(stepTime(), 100.0)) // cap time at 100sec
                    }

                    if(now > maxDuration) break

                    survivors.add(player)
                }
            }
        }
    }
}


Here,  we use both lazy injection with `inject<T>()` and instance retrieval with `get<T>()`. For details see [koin reference](https://doc.insert-koin.io/#/koin-core/injection-parameters)

The example is a true classic and was adopted from [salabim](https://github.com/salabim/salabim/blob/master/sample%20models/Gas%20station.py)'s and [SimPy](https://simpy.readthedocs.io/en/2.3.1/examples/gas_station_refuel.html)'s gas stations.


## Conclusion

In this example we have explored how a [depletable resource](../resource.md#depletable-resources) can be consumed by multipler clients.